Ilyes Justin m12001643 <br>
Seidl Stefan m11804717 <br>
Wagermaier Daniel m01605389

# Definition of input files

In [ ]:
#These filename need to be defined:
digits_train_filename = '../data/alldigits.csv'

# Calc1: Read data

In [ ]:
# --- Execution mandatory --- #
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# import Data
df = pd.read_csv(digits_train_filename)
data = df.to_numpy()

X = data[:, :-1]
X = X / 255.0 # normalize with max value
X_temp = []
for picture in X:
    X_temp.append(picture.reshape(28, 28).transpose())
X = np.array(X_temp)

y = data[:, -1]

labels = to_categorical(y, num_classes = 10)

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=42)

# Calc2: Convolutional Neural Network Classifier

### Manual Build of CNN

Starting off, we needed to get a feel for how our CNN would potentially be built. It was important to keep the model simple but also efficient. Overcomplicating the model could lead to unnecessarily long computational runtimes with results that are not even guarenteed to exceed the basic builds. 

In the beginning, we started off by testing models that had 2 Conv2d layers, both followed by MaxPooling2d layers, which then would be flattened and then ultimately passed to the output layer using an Adam optimizer (default learing rate). Initial attempts lead us to test accuracies around approximately 95-97%, which was without a doubt a modest starting point. This could definitely be optimised further. By experimenting with the filters in the Conv2d layers and their kernel sizes, we immediately saw increases and decreases in our results. After several attempts made, we implemented an additional fully-connected layer with a relu activation function following the flatten layer. This small change sufficiently helped our model reach higher accuracies consistently. To avoid overfitting of our model we threw in a dropout layer, which in our case would randomly set 50% of the previous dense layer's outputs to 0. After experimenting for a bit, we determined that it could possibly be beneficial to test out additional optimizers. We then realized that if the adadelta optimizer was used with higher learning rates it could challenge the effectiveness of the Adam optimizer. With us now using the Adadelta optimizer at a learning rate of 1, we continued to alternate through values in the parameters, and with a bit of luck, landed on a validation accuracy of around 98,5% in our model shown below.

In [ ]:
# --- Execution mandatory --- #
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
import tensorflow as tf

model = Sequential()

#model.add(data_augmentation)
model.add(Conv2D(64, kernel_size=(5,5), activation = 'relu', input_shape = (28, 28, 1)))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(128, kernel_size=(5,5), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

opt = tf.keras.optimizers.Adadelta(learning_rate=1)

model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics =['accuracy'])
model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 10, batch_size = 32)
# this came out to be 98,5% on one of its runs

# here we typically ended up in a range between 97,67% and 98,5%.

# Explore

### Convolutional Neural Network Classification: Hyperparameter Search

Our initial model of our CNN gave us an approximate idea of how our construct deciphered the data. Two things were very important to our newly discovered insights, it was possible to get high accuracies with simple models and it could also be done in a short amount of time. Now, it made sense to further our horizions and move on to more complex models that are not so timely effecient of us testing them. This meant a hyperparameter optimization was needed. The plan was, to keep the build relatively the same, maybe add a layer or two and change up the parameters in all of the layers to narrow down a possible advancement in the search for a more optimal model.

In [ ]:
import tensorflow as tf
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten

def build_model(hp):

    model = tf.keras.Sequential()

    previous_layer_filters = hp.Int('filter0', min_value = 64, max_value = 128, step = 32) # is useful, so that no following Conv2d layers are constructed with less filters than the previous one

    model.add(Conv2D(previous_layer_filters, 
                    kernel_size=hp.Int('kernel0', min_value = 3, max_value = 5, step = 2), 
                    activation = 'relu', 
                    input_shape = (28, 28, 1))) # first Conv2d layer receiving the input data
    model.add(MaxPooling2D(pool_size=(2,2))) # reduces the spatial size of the input

    layers = hp.Int('n_layers', min_value = 1, max_value = 2, step = 1) # so that different models with different layer amounts are created

    for i in range(layers): # additional Conv2d layers are added to the model
            previous_layer_filters = hp.Int(f'filter{i+1}', min_value = previous_layer_filters, max_value = 256, step = 32)
            model.add(Conv2D(previous_layer_filters, 
                            kernel_size=hp.Int(f'kernel{i+1}', min_value=3, max_value=5, step=2), 
                            activation = 'relu'))
    # we would have liked to include additional MaxPoolong2d layers into the for loop, but this lead to errors too often as the size of layers would become too small

    model.add(MaxPooling2D(pool_size=(2,2))) # so the MaxPooling2d layer was just added here instead

    if hp.Boolean('dropout_1'): # wanted to see if a dropout before the flatten layer would prove to be useful
        model.add(Dropout(hp.Float('dropout1', min_value = 0.25, max_value = 0.5, step_size = 0.25)))

    model.add(Flatten())

    model.add(Dense(hp.Int('dense', min_value = 64, max_value = 256, step = 64)))
    model.add(Activation('relu'))

    if hp.Boolean('dropout_2'):
        model.add(Dropout(hp.Float('dropout2', min_value = 0.25, max_value = 0.5, step_size = 0.25)))

    model.add(Dense(10)) # output
    model.add(Activation('softmax'))

    # going to test on the 2 optimizers that were most effecient in the manual experimenting phase before    
    hp_lr_adadelta = hp.Choice('learning_rate_adadelta', values = [1.5, 1.0, 0.5]) # higher learning rates were more effective for adadelta
    hp_lr_adam = hp.Choice('learning_rate_adam', values = [0.01, 0.001, 0.0001]) # and lower ones for adam 

    hp_optimizer = hp.Choice('optimizer', values = ['adadelta', 'adam'])

    if hp_optimizer == 'adadelta':
        optimizer = tf.keras.optimizers.Adadelta(learning_rate = hp_lr_adadelta)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate = hp_lr_adam)

    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics =['accuracy'])
    
    return model

#### Do hyperparameter search

To maximize our chances of finding more effectient models, we came to a resolution to use multiple hyperparameter optimization algorithms. The methods we used were RandomSearch, BayesianOptimization and Hyperband. Initially, only Hyperband was used as it was the fastest, but not always the most successful. Its usefulness lied in the fact that we could test out various ranges in the different parameters quickly and pin down the more useful parameters. After having reasonable values to alternate through our parameters, it made sense to try out the other 2 algorithms. These algorithms took much loner to run, but did prove to be rewarding.

In [ ]:
from keras_tuner import RandomSearch, BayesianOptimization, Hyperband

# using project_name and directory in the methods below, allows the user to save trials for later use. This is helpful for when the optimization is changed and the searches are restarted

tuner_rs = RandomSearch(build_model, objective='val_accuracy', max_trials=50, overwrite = False, project_name='RandomSearch', directory = 'Assignment_3/CNN_Clipboard')
tuner_rs.search(X_train, y_train, epochs=10, validation_data = (X_val, y_val))
# Define a search space as a bounded domain of hyperparameter values and randomly sample points in that domain (machinelearningmastery.com)

'''
'values': {'filter0': 96, 'kernel0': 5, 'n_layers': 2, 'filter1': 96, 'kernel1': 5, 'dropout_1': False, 'dense': 192, 'dropout_2': True, 'learning_rate_adadelta': 1.0, 
'learning_rate_adam': 0.0001, 'optimizer': 'adadelta', 'filter2': 160, 'kernel2': 3, 'dropout2': 0.5, 'dropout1': 0.25}

score: 0.9850000143051147
'''

tuner_bo = BayesianOptimization(build_model, objective='val_accuracy', max_trials=50, overwrite = False, project_name='BayesianOptimization', directory = 'Assignment_3/CNN_Clipboard')
tuner_bo.search(X_train, y_train, epochs=10, validation_data = (X_val, y_val))
#  keeps track of past evaluations which are then used in forming probabilistic models mapping their hyperparameters to a probability of a score to the objective

'''
'values': {'filter0': 64, 'kernel0': 5, 'n_layers': 1, 'filter1': 64, 'kernel1': 3, 'dropout_1': False, 'dense': 256, 'dropout_2': False, 
'learning_rate_adadelta': 1.0, 'learning_rate_adam': 0.0001, 'optimizer': 'adadelta', 'filter2': 256, 'kernel2': 5, 'dropout1': 0.5, 'dropout2': 0.25

score: 0.9850000143051147
'''


tuner_hb = Hyperband(build_model, objective='val_accuracy', max_epochs=12, overwrite = False, project_name='Hyperband', directory = 'Assignment_3/CNN_Clipboard')
tuner_hb.search(X_train, y_train, epochs=10, validation_data = (X_val, y_val))
# similar to a sports-style playoff bracket. At first goes through 2 epochs per trial, only keeps the best and does 4 epochs of training on them and then the same for 12 epochs again

'''
'values': {'filter0': 64, 'kernel0': 3, 'n_layers': 2, 'filter1': 256, 'kernel1': 5, 'dropout_1': True, 'dense': 128, 'dropout_2': False, 
'learning_rate_adadelta': 0.1, 'learning_rate_adam': 0.001, 'optimizer': 'adam', 'tuner/epochs': 12, 'tuner/initial_epoch': 4, 
'tuner/bracket': 2, 'tuner/round': 2, 'filter2': 256, 'kernel2': 3, 'dropout1': 0.25

score: 0.9783333539962769
'''

#### Hyperparameter Optimization Results:

After running the various algorithms we recieved 2 models that returned identical validation accuracies of 98,5%. This was a bit disappointing, because this meant that our model did not improve after the hyperparameter optimization. Ultimately, only the runtimes had increased and out of that reason we could infer that the simpler model was still more optimal than the more complex ones. This lead us to look at the problem alternatively in hopes of still possibly finding a method to accrue improvements in our efforts. 

After thoroughly searching through different concepts that might improve our results, we came across image augmentation. This method takes input data (our images of digits) and can manipulate these images in a various ways. There are only a few methods though that can be useful when it comes to digits, like for instance the method RandomRotation where the image is randomly rotated by a small degree and then used as input for the model. Other methods however, like RandomFlip return much worse accuracies, which was not too surprising, as it would not make sense to invert an image of a number horizontally or vertically, because that would not accurately depict the number anymore. An 8 upsidedown is still an 8, but a 7 upsidedown is absolutely nothin at all. All this experimenting gave us no additional insight of how to better our model, because it did not lead to any optimised results in the end. Surprisingly enough, randomly rotation the digits just made things worse.

### Estimation of accuracy

To calculate the empirical accuracy of the model the function 'evaluate()' was used. This function returns the loss value and the accuracy of the model. The accuracy is calculated by comparing the predicted labels with the true labels. The accuracy is the number of correct predictions divided by the total number of input samples. <br>

To calculate the lower precision bound the following formula was used: <br>
$$ 
\varepsilon \leq 
    \hat{\varepsilon} - c_{\delta} ∗ \sqrt{
    \frac{ \hat{\varepsilon} ∗ (1 − \hat{\varepsilon})}{n}}  
    - \frac{c^2_{\delta}}{n} 
$$
 
$\hat{\varepsilon}$ ...empirical error or empirical precision <br>
$\varepsilon$   ...true error or true precision <br>
$n$ ...number of samples <br>
$c_{\delta}$  ...quantile of normal distribution <br>

In [ ]:
from scipy import stats

loss, accuracy = model.evaluate(X_test, y_test)
significance_level = 0.05
c_delta = stats.norm.ppf(1-significance_level)

lower_bound_accuracy = accuracy - c_delta * np.sqrt((accuracy * (1 - accuracy)) / len(y_test)) - c_delta ** 2 / len(y_test)
print(f'''
    CNN Classifier:
    empirical accuracy: {round(accuracy * 100, 2)}%
    lower accuracy bound: {round(lower_bound_accuracy * 100, 2)}%''')

print(f'\nRegarding the accuracy of the CNN classifier, we can state that the precision of our classifier is at least {round(lower_bound_accuracy * 100, 2)}% in {round((1 - significance_level) * 100, 2)}% of the cases.')

# Final classifier

In [ ]:
digits_CNN = model